In [ ]:
checkpoints==

In [ ]:
from api_helper import ShoonyaApiPy
import logging
import time
import pandas as pd
import json
import numpy as np
import cred
from datetime import datetime
import time
from notify_run import Notify
notify = Notify()
import pyotp
token = cred.token
totp = pyotp.TOTP(token).now()
#enable dbug to see request and responses
logging.basicConfig(level=logging.DEBUG)
#start of our program
api = ShoonyaApiPy()
#credentials
user = cred.user
pwd = cred.pwd
factor2 = totp
vc = cred.vc
app_key = cred.apikey
imei = cred.imei
logins = api.login(userid=user, password=pwd, twoFA=factor2, vendor_code=vc, 
    api_secret=app_key,imei = imei)
print(logins)

In [ ]:
api.

In [ ]:
df = []

In [ ]:
a = api.get_order_book()

In [ ]:
a

In [ ]:
for pp in range(len(a)):
    if((a[pp]['trantype']=='B')&(a[pp]['status']=='OPEN') ):
        print('yep')
        break

In [ ]:
api.get_quotes('NFO','NIFTY18MAY23C18500')['bp1']

In [ ]:
a[0]['norenordno']

In [ ]:
api.cancel_order(a[0]['norenordno'])

In [ ]:
totp

In [ ]:
df = []
for i in range(40):
    df.append(pd.DataFrame(columns = ['bq1','bp1','sp1','sq1','v']))

In [ ]:
api.get_quotes('NFO',a)['bp1']

In [ ]:
b = api.searchscrip('NFO','NIFTY18MAY23C18500')
a= b['values'][0]['token']

In [ ]:
api.single_order_history

In [ ]:
sq=sp=bq=bp=v=0
feed_opened = False

In [ ]:

def event_handler_feed_update(m):
    print(m)
    global feed_opened,df,a,sq,sp,bq,bp,v
    
    ind = np.where(a==m['tk'])[0][0]
    
    if('sq1'in m):
        sq = float(m['sq1'])
    else:
        sq = df[ind]['sq1'][df[ind]['sq1'].count()-1]
        
    if('sp1'in m):
        sp = float(m['sp1'])
    else:
        sp = df[ind]['sp1'][df[ind]['sp1'].count()-1]
    if('bq1'in m):
        bq = float(m['bq1'])
    else:
        bq = df[ind]['bq1'][df[ind]['bq1'].count()-1]
    if('bp1'in m):
        bp = float(m['bp1'])
    else:
        bp = df[ind]['bp1'][df[ind]['bp1'].count()-1]
    if('v'in m):
        v = float(m['v'])
    else:
        v = df[ind]['v'][df[ind]['v'].count()-1]
    
    df[ind].loc[df[ind]['v'].count()] = [bq,bp,sp,sq,v]
    
    
    
    
        
def event_handler_order_update(inmessage):
    print(inmessage)                                      #     if(('norennordno' in inmessage)&('status'in inmessage)):  #         orderJson[inmessage['norenordno']] = {'status': inmessage['status']}
        
def open_callback():
    
    global feed_opened
    feed_opened = True



In [ ]:
api.start_websocket( order_update_callback=event_handler_order_update,
 subscribe_callback=event_handler_feed_update, 
socket_open_callback=open_callback)
while(feed_opened==False):
    pass


In [ ]:
feed_opened

In [ ]:
#api.close_websocket()

In [ ]:
a =np.zeros(40)
x = 18500
for i in range(20):
    
    b = api.searchscrip('NFO','NIFTY11MAY23C'+str(x))
    a[i] = b['values'][0]['token']
    x = x+50
    
x = 18100
for i in range(20):
    
    b = api.searchscrip('NFO','NIFTY11MAY23P'+str(x))
    a[20+i] = b['values'][0]['token']
    x = x-50


In [ ]:
a

In [ ]:
b = []
for i in range(40):
    b.append('NFO|'+str(int(a[i])))
a = a.astype(int)
a = a.astype(str)

In [ ]:
a

In [ ]:
api.subscribe(b)

In [ ]:
df[0]

In [ ]:
from datetime import datetime, timedelta
import time

In [ ]:
curr = datetime.now()
new = curr+timedelta(seconds = 70)

In [ ]:
while(1):
    while(curr.minute != new.minute):
        curr = datetime.now()
    api.unsubscribe(b)
    api.close_websocket()
    
    feed_opened = False
    time.sleep(1)
    
    api.start_websocket( order_update_callback=event_handler_order_update, subscribe_callback=event_handler_feed_update, socket_open_callback=open_callback)
    while(feed_opened==False):
        pass
    time.sleep(1)
    api.subscribe(b)
    
    new = curr + timedelta(minutes = 2)
    

In [ ]:
feed_opened

In [ ]:
df[0]

In [ ]:
for i in range(40):
    df[i].to_csv('thu'+str(i)+'.csv',index=True)